<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250703%5D%20TrustFed-IDS%20%E2%80%93%20WSN-BFSF%20Ablation%20sweep%20(30%20rounds%20each).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TrustFed-IDS – WSN-BFSF  |  Ablation sweep (30 rounds each)**

In [24]:
# --------------------------------------------------------------
#  TrustFed-IDS – WSN-BFSF  |  Ablation sweep (30 rounds each)
# --------------------------------------------------------------
import os, time, psutil, numpy as np, pandas as pd, tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.schedules import CosineDecay
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from scipy.spatial.distance import cosine

In [25]:
# ----------------------- fixed experiment settings -----------------------
SEED            = 42
NUM_CLIENTS     = 5
ROUNDS          = 30             # ← ablation uses 30 rounds
LOCAL_EPOCHS    = 1
BATCH_SIZE      = 32
DIRICHLET_ALPHA = 0.5
HISTORY_KEEP    = 6
LOG_DIR         = "/mnt/data"
DATA_PATH       = "dataset.csv"   # ← make sure the CSV is here
os.makedirs(LOG_DIR, exist_ok=True)

np.random.seed(SEED); tf.random.set_seed(SEED)

In [26]:
# -------------------------- load + encode dataset ------------------------
df = pd.read_csv(DATA_PATH).dropna()
for col in df.select_dtypes(include="object"):
    df[col] = LabelEncoder().fit_transform(df[col])

X_all = df.drop("Class", axis=1).values.astype("float32")
y_all = df["Class"].values.astype("int64")
num_classes = int(y_all.max() + 1)

In [27]:
# ----------------- helper: Dirichlet non-IID partition -------------------
def dirichlet_split(X, y, k, alpha, rng):
    idx_by_cls = {c: rng.permutation(np.where(y == c)[0]) for c in np.unique(y)}
    clients = [[] for _ in range(k)]
    for c, idx in idx_by_cls.items():
        parts = (rng.dirichlet([alpha]*k) * len(idx)).astype(int)
        while parts.sum() < len(idx):
            parts[rng.randint(0, k)] += 1
        s = 0
        for cid, ct in enumerate(parts):
            clients[cid].extend(idx[s:s+ct]); s += ct
    for lst in clients: rng.shuffle(lst)
    return [X[l] for l in clients], [y[l] for l in clients]

In [28]:
# -------------------------- model factory --------------------------------
def build_model(input_shape, classes):
    lr = CosineDecay(5e-4, decay_steps=ROUNDS, alpha=0.4)
    opt = tf.keras.optimizers.Nadam(lr, clipnorm=2.0)
    m = Sequential([
        InputLayer(shape=input_shape),
        LSTM(128, return_sequences=True, activation='tanh',
             kernel_regularizer=l2(5e-4)),
        LSTM(64, activation='tanh', kernel_regularizer=l2(5e-4)),
        Dense(256, activation='relu'), Dropout(0.20),
        Dense(128, activation='relu'), Dropout(0.25),
        Dense(classes, activation='softmax')
    ])
    m.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return m

In [29]:
# ----------------------- trust & aggregation utils -----------------------
def weight_delta(local, global_):
    return [l - g for l, g in zip(local, global_)]
def vec_cos(a, b):
    v1, v2 = np.concatenate([w.ravel() for w in a]), np.concatenate([w.ravel() for w in b])
    if np.all(v1 == 0) or np.all(v2 == 0): return 0.0
    return 1 - cosine(v1, v2)
def stability(u, hist):
    if len(hist) < 2: return 1.0
    return float(np.mean([vec_cos(u, h) for h in hist[-HISTORY_KEEP:]]))
def compute_trust(upd, vloss, hist, alpha):
    lo, hi = min(vloss.values()), max(vloss.values())
    trust = {}
    for cid, u in upd.items():
        sim  = vec_cos(u, [np.zeros_like(w) for w in u])
        loss = 1 - (vloss[cid] - lo) / (hi - lo + 1e-8)
        stab = stability(u, hist[cid])
        trust[cid] = max(alpha[0]*sim + alpha[1]*loss + alpha[2]*stab, 1e-6)
    return trust
def aggregate(ws, t, n):
    tot = sum(t[c]*n[c] for c in ws)
    return [sum(t[c]*n[c]*ws[c][l] for c in ws)/tot
            for l in range(len(next(iter(ws.values()))))]

In [30]:
# ------------------------ ablation variants ------------------------------
VARIANTS = {
    "baseline":   dict(use_smote=True,  alpha=(0.30,0.55,0.15), clip=(0.05,0.60)),
    "no_smote":   dict(use_smote=False, alpha=(0.30,0.55,0.15), clip=(0.05,0.60)),
    "weight_swap":dict(use_smote=True,  alpha=(0.50,0.35,0.15), clip=(0.05,0.60)),
    "clip_0.50":  dict(use_smote=True,  alpha=(0.30,0.55,0.15), clip=(0.05,0.50)),
}

In [31]:
# ======================= main loop over variants =========================
for tag, cfg in VARIANTS.items():
    print(f"\n=== Running variant: {tag} ===")
    # 1) split train / test
    X_tr, X_te, y_tr, y_te = train_test_split(
        X_all, y_all, test_size=0.20, stratify=y_all, random_state=SEED)
    scaler = StandardScaler().fit(X_tr)
    X_tr, X_te = scaler.transform(X_tr), scaler.transform(X_te)

    # optional SMOTE
    if cfg["use_smote"]:
        X_tr, y_tr = SMOTE(random_state=SEED).fit_resample(X_tr, y_tr)

    # reshape for LSTM
    X_tr, X_te = X_tr[..., None], X_te[..., None]
    y_te_cat = to_categorical(y_te, num_classes)

    # 2) non-IID Dirichlet split (after balancing so each client sees same size)
    rng = np.random.RandomState(SEED)
    cX_raw, cy_raw = dirichlet_split(X_tr, y_tr, NUM_CLIENTS,
                                     DIRICHLET_ALPHA, rng)
    client_X = cX_raw
    client_y = [to_categorical(y, num_classes) for y in cy_raw]

    # 3) initialise global model
    g_model   = build_model(input_shape=(X_tr.shape[1],1), classes=num_classes)
    g_weights = g_model.get_weights()
    model_MB  = sum(w.nbytes for w in g_weights)/2**20

    histories = {c: [] for c in range(NUM_CLIENTS)}
    perf_log, comm_log, trust_log = [], [], []

    cls_wt = compute_class_weight('balanced',
                                  classes=np.arange(num_classes), y=y_tr)
    class_weight = dict(enumerate(cls_wt))

    # 4) federated rounds
    for r in range(1, ROUNDS+1):
        tic = time.time()
        lw, upd, vloss, ns, bytes_out = {}, {}, {}, {}, 0
        for cid in range(NUM_CLIENTS):
            n_val = max(1, int(0.1*len(client_X[cid])))
            Xv, yv = client_X[cid][:n_val], client_y[cid][:n_val]
            Xt, yt = client_X[cid][n_val:], client_y[cid][n_val:]

            local = build_model(input_shape=(X_tr.shape[1],1), classes=num_classes)
            local.set_weights(g_weights)
            local.fit(Xt, yt, epochs=LOCAL_EPOCHS, batch_size=BATCH_SIZE,
                      verbose=0, class_weight=class_weight)

            w = local.get_weights()
            u = weight_delta(w, g_weights)
            l = local.evaluate(Xv, yv, verbose=0)[0]
            lw[cid], upd[cid], vloss[cid], ns[cid] = w, u, l, len(Xt)
            histories[cid] = (histories[cid] + [u])[-HISTORY_KEEP:]
            bytes_out += sum(x.nbytes for x in w)

        trust = compute_trust(upd, vloss, histories, cfg["alpha"])
        # clip & renormalise
        tmin, tmax = cfg["clip"]
        trust = {c: np.clip(s, tmin, tmax) for c,s in trust.items()}
        Z = sum(trust.values())
        trust = {c: s/Z for c,s in trust.items()}

        g_weights = aggregate(lw, trust, ns)
        g_model.set_weights(g_weights)

        # ---- evaluation on test set ----
        y_pred = np.argmax(g_model.predict(X_te, verbose=0), axis=1)
        perf_log.append({
            "round": r,
            "accuracy":  accuracy_score(y_te, y_pred),
            "precision": precision_score(y_te, y_pred, average='weighted', zero_division=0),
            "recall":    recall_score(y_te, y_pred, average='weighted', zero_division=0),
            "f1":        f1_score(y_te, y_pred, average='weighted', zero_division=0),
            "ms": round((time.time()-tic)*1000, 2)
        })
        comm_log.append({"round": r, "MB": bytes_out/2**20})
        trust_log.extend([{"round": r, "client": c, "trust": trust[c]}
                          for c in trust])

        print(f"[{tag}] R{r:02d}  acc={perf_log[-1]['accuracy']:.8f}  "
              f"F1={perf_log[-1]['f1']:.8f}"
              f"P={perf_log[-1]['precision']:.8f}"
              f"R={perf_log[-1]['recall']:.8f}")


=== Running variant: baseline ===
[baseline] R01  acc=0.57154529  F1=0.65495827P=0.85708010R=0.57154529
[baseline] R02  acc=0.63778796  F1=0.73200241P=0.91280663R=0.63778796
[baseline] R03  acc=0.78928903  F1=0.83944433P=0.93002616R=0.78928903
[baseline] R04  acc=0.76378520  F1=0.82170739P=0.93376543R=0.76378520
[baseline] R05  acc=0.78382622  F1=0.83543322P=0.93382520R=0.78382622
[baseline] R06  acc=0.78872833  F1=0.83897087P=0.93932934R=0.78872833
[baseline] R07  acc=0.82639134  F1=0.86604780P=0.94292608R=0.82639134
[baseline] R08  acc=0.82885842  F1=0.86800069P=0.94376175R=0.82885842
[baseline] R09  acc=0.84550319  F1=0.87887268P=0.94114935R=0.84550319
[baseline] R10  acc=0.82663164  F1=0.86653893P=0.94361836R=0.82663164
[baseline] R11  acc=0.83954375  F1=0.87477499P=0.93970228R=0.83954375
[baseline] R12  acc=0.81850950  F1=0.86043130P=0.93910303R=0.81850950
[baseline] R13  acc=0.83688443  F1=0.87305121P=0.94106068R=0.83688443
[baseline] R14  acc=0.82741662  F1=0.86629779P=0.936770

In [32]:
# ---------------- 5. SAVE LOGS (variant-specific filenames) -------------
suffix = tag          # e.g. baseline / no_smote / weight_swap / clip_0.50

pd.DataFrame(perf_log ).to_csv(f"{LOG_DIR}/perf_log_WSN-BFSF_{suffix}.csv",
                               index=False)
pd.DataFrame(comm_log ).to_csv(f"{LOG_DIR}/comm_log_WSN-BFSF_{suffix}.csv",
                               index=False)
pd.DataFrame(trust_log).to_csv(f"{LOG_DIR}/trust_log_WSN-BFSF_{suffix}.csv",
                               index=False)

model_profile = dict(
    Params_MB   = round(model_MB, 3),
    Rounds      = ROUNDS,
    Clients     = NUM_CLIENTS,
    PeakMem_MB  = round(psutil.Process(os.getpid()).memory_info().rss / 2**20, 2)
)
pd.DataFrame([model_profile]).to_csv(
    f"{LOG_DIR}/model_profile_WSN-BFSF_{suffix}.csv",
    index=False
)

print(f"✓ Variant '{suffix}' complete — logs saved to {LOG_DIR}")

✓ Variant 'clip_0.50' complete — logs saved to /mnt/data


In [33]:
import glob
import os

log_files_path = "/mnt/data/"
log_files = glob.glob(os.path.join(log_files_path, "*.csv"))

print("Generated log files:")
for f in log_files:
    print(os.path.basename(f))

Generated log files:
trust_log_WSN-BFSF_0.csv
0_perf.csv
0_comm.csv
perf_log_WSN-BFSF_0.csv
perf_log_WSN-BFSF_clip_0.50.csv
comm_log_WSN-BFSF_0.csv
0_trust.csv
model_profile_WSN-BFSF_0.csv
trust_log_WSN-BFSF_clip_0.50.csv
comm_log_WSN-BFSF_clip_0.50.csv
model_profile_WSN-BFSF_clip_0.50.csv


In [34]:
import pandas as pd, glob
for f in glob.glob('/mnt/data/perf_log_WSN-BFSF_*.csv'):
    tag = f.split('_')[-1].split('.')[0]   # baseline / no_smote / ...
    df  = pd.read_csv(f)
    print(tag, df[['accuracy','f1']].tail(1).values)

0 [[0.46142386 0.5452185 ]]
0 [[0.87014194 0.89560421]]
